In [1]:
import pandas as pd

# Load the CSV file
file_path = 'data/student_data_unprocessed.csv'
df = pd.read_csv(file_path)

# Display the first few rows to understand the structure of the data
df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'data/student_data_unprocessed.csv'

In [5]:
# Step 1: Create the required columns
output_df = pd.DataFrame()

# List of startups in current cohort - MUST match the order they appear in startups_data sheet
startup_names = [
'BreakBio Corp', 'CartaBio', 'Wright Electric', 'DataHive', '42Genetics', 'SOAP Health', 'Cyanotype Bio', 'Free Form Fibers', 'MabSwitch', 
'Hoox', 'Elm AI', 'Godot', 'ZayaAI', 'Telness Tech', 'Smabbler','CAREier', 'PathKeeper Surgical', 'GrayMatters Health', 'SyntheX',
'Helix Earth', 'AdviNOW Medical', 'Sensible Biotechnologies', 'Modulate Bio', 'Torpedo Therapeutics', 'IDP Pharma', 'Werewool', 'Rasayana Therapeutics',
'Cellinfinity Bio', 'dataspan.ai', 'Libera Bio', 'RealNose.ai', 'Simulacra SDS', 'Evvy', 'Atmo Biosciences', 'Avol', 'LATERAL.systems', 'Coastar Therapeutics', 
'Rejuvenation Technologies', 'Ceramedix', 'MiCare Path', 'Picture Health', 'BlueWhale Bio', 'Siftwell Analytics', 'DoriNano', 'Script Biosciences', 
'Relyion Energy', 'Olfera', 'RISE Robotics', 'Lore Machine', 'Fermbox Bio', 'SEED Therapeutics', 'Revolv3', 'Nest Genomics', 'Frost Methane', 
'Sama Therapeutics', 'CTGT', 'REEV', 'Serentrix', 'NavaFlex', 'ByteGenie', 'Koi Biotherapeutics', 'CLIKA', 'Cloverleaf Bio', 'Sunchem', 'Robochef', 
'OLLANG', 'Luxsonic', 'Tessel Bio', 'SpruceID', 'Plantibodies', 'PLATMA', 'ExoDiscovery', 'Suyana Climate Insurance', 'Stimvia', 'Apricity Health', 'iLoF', 
'ATOMICS', 'Banquet Health', 'nSight Surgical', 'Banyan Infrastructure', 'Vitro3D', 'Solvuu', 'The Rubic', 'Agilis Commerce', 
'Trust Science', 'syd.life', 'Enigma Aerospace', 'Page Technologies', 'Safe YOU | Impact Innovations Institute', 'Verne', 'geCKo Materials', 'Schnell LiFi Germany',
'LUCID', 'Merlin Biotech', 'Propitious Technologies', 'Youverify', 'WORMSENSING', 'VoicePlug', 'Fenix Commerce'
]


# Extract student name
output_df['student name'] = df['Name']

# Track rankings columns (Deep Tech, Digital Tech, Life Sciences, Digital Health)
# These are scaled to a range of 0.25 to 1 based on the rank value
def scale_ranking(rank):
    return 1.25 - (0.25 * rank)

output_df['Deep tech rank'] = df['Deep Tech track ranking'].apply(scale_ranking)
output_df['Digital tech rank'] = df['Digital Tech track ranking'].apply(scale_ranking)
output_df['Life sciences rank'] = df['Life Sciences track ranking'].apply(scale_ranking)
output_df['Digital health rank'] = df['Digital Health track ranking'].apply(scale_ranking)

# Step 2: Normalize skill values by dividing by 7 (assuming certain columns correspond to skills)
# Columns containing skill values (manual inspection required based on column names)
skill_columns = [
    'Evaluating Potential Markets/Choosing Target Market',
    'Developing Pricing Model',
    'Customer Decision Journey',
    'Pilot Project Design and Implementation',
    'Investment Pitch Deck',
    'Hiring Plan',
    'Marketing Plan & Execution',
    'Business Development & Sales',
    'Scaling Operations'
]
for col in skill_columns:
    output_df[col] = df[col] / 7

# Step 3: Generate startup preference columns
# Assuming preference columns are under the following structure:
preference_columns = [
    'Deep Tech Companies Preferences',
    'Digital Tech Companies Preferences',
    'Life Sciences Companies Preferences',
    'Digital Health Companies Preferences'
]

# Create columns for each startup
for startup in startup_names:
    output_df[startup] = 0.1  # Default value

# Function to assign weights based on preference rank
def assign_weights(pref_list, no_of_companies):
    weights = {}
    for idx, company in enumerate(pref_list):
        weight = (no_of_companies - idx) * (1 / no_of_companies)
        weights[company.strip()] = weight
    return weights

# Fill the startup preferences with scaled weights based on the student's preference
for i, row in df.iterrows():
    for col in preference_columns:
        if row[col]:
            company_list = row[col].split(',')
            company_weights = assign_weights(company_list, len(company_list))
            for company, weight in company_weights.items():
                if company in output_df.columns:
                    output_df.at[i, company] = weight

# Pulling matched students from the algo
student_names_remove = ['Zhang, Hui', 'Hayden, Nicholas', 'Zhai, Danni', 'Anifa Saitbai, Abraar']
filtered_df = output_df[~output_df['student name'].isin(student_names_remove)]

# Round all floating point values to 2 decimal places
filtered_df = filtered_df.round(2)

# Save the updated output to CSV
output_file_rounded = 'data/processed_student_data.csv'
filtered_df.to_csv(output_file_rounded, index=False)

output_file_rounded

/var/folders/0f/yysl65fd0r91jjvmdt0jr5qw0000gn/T/ipykernel_36875/2290472326.py:60: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[startup] = 0.1  # Default value
/var/folders/0f/yysl65fd0r91jjvmdt0jr5qw0000gn/T/ipykernel_36875/2290472326.py:60: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_df[startup] = 0.1  # Default value
/var/folders/0f/yysl65fd0r91jjvmdt0jr5qw0000gn/T/ipykernel_36875/2290472326.py:60: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` man

'data/processed_student_data.csv'

In [6]:
output_df.head

<bound method NDFrame.head of                student name  Deep tech rank  Digital tech rank  \
0            Chang, Wei Lin            0.50               0.25   
1           McDougall, Dawn            0.75               0.50   
2            Gradelski, Mia            1.00               0.75   
3             Lahanis, Niko            1.00               0.50   
4            Johnston, Maya            0.75               0.50   
..                      ...             ...                ...   
97       Berdegué, Santiago            1.00               0.50   
98              Dai, Qinqin            0.75               0.25   
99              Tang, Derek            1.00               0.75   
100       Chauhan, Abhyudai            1.00               0.75   
101  Sauter Tamayo, Kristel            0.25               0.50   

     Life sciences rank  Digital health rank  \
0                  0.75                 1.00   
1                  1.00                 0.25   
2                  0.50          